In [1]:
import time
import numpy as np
import pandas as pd
import catboost as cb

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, train_test_split

### functions

In [2]:
def get_lower(x: str) -> str:
    return x.replace(' ', '_').lower()

def one_hot_encoding(df, cat_features=None, drop_original=True):
    df_copy = df.copy()
    if cat_features is None:
        cat_features = df.columns
    for feature in cat_features:
        df_copy = pd.concat([df_copy, 
                             pd.get_dummies(df[feature], 
                                            prefix=feature, 
                                            dummy_na=True)
                            ],
                            axis=1)
    if drop_original:
        df_copy.drop(cat_features, axis=1, inplace=True)
    return df_copy

def agg_features(df, df_out, by: str):
    df_copy = df.copy()
    df_out_copy = df_out.copy()
    cols = df_copy.drop(by, axis=1).columns
    for feature in cols:
        feature_groupby = df_copy.groupby(by, as_index=False)
        feature_counts = feature_groupby[feature].mean()
        feature_counts = feature_counts.rename(columns={
            feature: feature + f'_mean'
        })
        df_out_copy = df_out_copy.merge(feature_counts, how="left", on=by)
    return df_out_copy

def get_input(data_path: str,
              base_path: str= './data/',
              col_lower=True) -> pd.DataFrame:
    data = pd.read_csv(f'{base_path}/{data_path}')
    if col_lower:
        data.columns = [col.lower() for col in data.columns]
    print(f'{data_path}: {data.shape[0]} rows, {data.shape[1]} cols')
    return data

### base tables

In [3]:
train = get_input('train.csv')
test = get_input('test.csv')

data = pd.concat([train, test], axis=0)
data = data.reset_index(drop=True)
data.head(3)

train.csv: 110093 rows, 3 cols
test.csv: 165141 rows, 2 cols


,application_number,target,name_contract_type
0,123687442,0.0,Cash
1,123597908,1.0,Cash
2,123526683,0.0,Cash


In [4]:
target_name = 'target'

In [5]:
data['name_contract_type'].value_counts()

Cash           248983
Credit Card     26251
Name: name_contract_type, dtype: int64

In [6]:
data = one_hot_encoding(data, 
                        cat_features=['name_contract_type'], 
                        drop_original=True)

In [7]:
data.head(3)

,application_number,target,name_contract_type_Cash,name_contract_type_Credit Card,name_contract_type_nan
0,123687442,0.0,1,0,0
1,123597908,1.0,1,0,0
2,123526683,0.0,1,0,0


### client profile

In [8]:
profile = get_input('client_profile.csv')
profile.head(3)

client_profile.csv: 250000 rows, 24 cols


,application_number,gender,childrens,total_salary,amount_credit,amount_annuity,education_level,family_status,region_population,age,...,family_size,external_scoring_rating_1,external_scoring_rating_2,external_scoring_rating_3,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
0,123666076,F,0,157500.0,270000.0,13500.0,Incomplete higher,Civil marriage,0.008068,8560,...,2.0,0.329471,0.236315,0.678568,0.0,0.0,0.0,0.0,1.0,2.0
1,123423688,F,0,270000.0,536917.5,28467.0,Secondary / secondary special,Married,0.020246,23187,...,2.0,NaN,0.442295,0.802745,0.0,0.0,0.0,0.0,1.0,1.0
2,123501780,M,1,427500.0,239850.0,23850.0,Incomplete higher,Married,0.072508,14387,...,3.0,0.409017,0.738159,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
numerical_features = profile.select_dtypes(exclude=["object"]).columns.tolist()
categorical_features = profile.select_dtypes(include=["object"]).columns.tolist()

In [10]:
categorical_features

['gender', 'education_level', 'family_status']

In [11]:
for feature in profile[categorical_features].columns.tolist():
    print()
    print(profile[feature].value_counts())


F      164645
M       85351
XNA         4
Name: gender, dtype: int64

Secondary / secondary special    177562
Higher education                  60864
Incomplete higher                  8357
Lower secondary                    3081
Academic degree                     136
Name: education_level, dtype: int64

Married                 159846
Single / not married     36883
Civil marriage           24252
Separated                15984
Widow                    13033
Unknown                      2
Name: family_status, dtype: int64


In [12]:
profile_ohe = one_hot_encoding(profile, 
                               cat_features=categorical_features, 
                               drop_original=True)

In [13]:
data = data.merge(profile_ohe, how="left", on='application_number')

In [14]:
data.head(3)

,application_number,target,name_contract_type_Cash,name_contract_type_Credit Card,name_contract_type_nan,childrens,total_salary,amount_credit,amount_annuity,region_population,...,education_level_Lower secondary,education_level_Secondary / secondary special,education_level_nan,family_status_Civil marriage,family_status_Married,family_status_Separated,family_status_Single / not married,family_status_Unknown,family_status_Widow,family_status_nan
0,123687442,0.0,1,0,0,1.0,157500.0,855000.0,25128.0,0.019101,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,123597908,1.0,1,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,123526683,0.0,1,0,0,0.0,135000.0,1006920.0,42660.0,0.026392,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [15]:
data.shape

(275234, 42)

### applications history

In [16]:
app_history = get_input('applications_history.csv')
app_history.head(3)

applications_history.csv: 1670214 rows, 26 cols


,prev_application_number,application_number,name_contract_type,amount_annuity,amt_application,amount_credit,amount_payment,amount_goods_payment,name_contract_status,days_decision,...,name_product_type,sellerplace_area,cnt_payment,name_yield_group,days_first_drawing,days_first_due,days_last_due_1st_version,days_last_due,days_termination,nflag_insured_on_approval
0,49298709,123595216,NaN,1730.430,17145.0,17145.0,0.0,17145.0,Approved,73,...,XNA,35,12.0,middle,365243.0,42.0,300.0,42.0,37.0,0.0
1,50070639,123431468,Cash,25188.615,607500.0,679671.0,NaN,607500.0,Approved,164,...,x-sell,-1,36.0,low_action,365243.0,134.0,916.0,365243.0,365243.0,1.0
2,49791680,123445379,Cash,15060.735,112500.0,136444.5,NaN,112500.0,Approved,301,...,x-sell,-1,12.0,high,365243.0,271.0,59.0,365243.0,365243.0,1.0


In [17]:
numerical_features = app_history.select_dtypes(exclude=["object"]).columns.tolist()
categorical_features = app_history.select_dtypes(include=["object"]).columns.tolist()

In [18]:
categorical_features

['name_contract_type',
 'name_contract_status',
 'name_payment_type',
 'code_reject_reason',
 'name_type_suite',
 'name_client_type',
 'name_goods_category',
 'name_portfolio',
 'name_product_type',
 'name_yield_group']

In [19]:
for feature in app_history[categorical_features].columns.tolist():
    print()
    print(app_history[feature].value_counts())


Cash           747553
Credit Card    193164
Name: name_contract_type, dtype: int64

Approved        1036781
Canceled         316319
Refused          290678
Unused offer      26436
Name: name_contract_status, dtype: int64

Cash through the bank                        1033552
XNA                                           627384
Non-cash from your account                      8193
Cashless from the account of the employer       1085
Name: name_payment_type, dtype: int64

XAP       1353093
HC         175231
LIMIT       55680
SCO         37467
CLIENT      26436
SCOFR       12811
XNA          5244
VERIF        3535
SYSTEM        717
Name: code_reject_reason, dtype: int64

Unaccompanied      508970
Family             213263
Spouse, partner     67069
Children            31566
Other_B             17624
Other_A              9077
Group of people      2240
Name: name_type_suite, dtype: int64

Repeater     1231261
New           301363
Refreshed     135649
XNA             1941
Name: name_client_typ

In [20]:
app_history_ohe = one_hot_encoding(app_history, 
                                   cat_features=categorical_features, 
                                   drop_original=True)

In [21]:
data = agg_features(app_history_ohe, data, by='application_number')

In [22]:
data.head(3)

,application_number,target,name_contract_type_Cash,name_contract_type_Credit Card,name_contract_type_nan,childrens,total_salary,amount_credit,amount_annuity,region_population,...,name_product_type_XNA_mean,name_product_type_walk-in_mean,name_product_type_x-sell_mean,name_product_type_nan_mean,name_yield_group_XNA_mean,name_yield_group_high_mean,name_yield_group_low_action_mean,name_yield_group_low_normal_mean,name_yield_group_middle_mean,name_yield_group_nan_mean
0,123687442,0.0,1,0,0,1.0,157500.0,855000.0,25128.0,0.019101,...,1.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.000000,0.333333,0.0
1,123597908,1.0,1,0,0,NaN,NaN,NaN,NaN,NaN,...,0.5,0.0,0.5,0.0,0.0,0.750000,0.250000,0.000000,0.000000,0.0
2,123526683,0.0,1,0,0,0.0,135000.0,1006920.0,42660.0,0.026392,...,0.5,0.0,0.5,0.0,0.5,0.166667,0.000000,0.166667,0.166667,0.0


In [23]:
data.shape

(275234, 138)

### payments

In [24]:
payments = get_input('payments.csv')
payments.head(3)

payments.csv: 1023932 rows, 8 cols


,prev_application_number,application_number,num_instalment_version,num_instalment_number,days_instalment,days_entry_payment,amt_instalment,amt_payment
0,49011181,123664960,1.0,5,1002.0,1015.0,12156.615,12156.615
1,48683432,123497205,1.0,13,442.0,432.0,18392.535,10047.645
2,48652024,123749925,1.0,10,8.0,23.0,5499.945,5499.945


In [25]:
numerical_features = payments.select_dtypes(exclude=["object"]).columns.tolist()
categorical_features = payments.select_dtypes(include=["object"]).columns.tolist()

In [26]:
categorical_features

[]

In [27]:
data = agg_features(payments, data, by='application_number')

In [28]:
data.head(3)

,application_number,target,name_contract_type_Cash,name_contract_type_Credit Card,name_contract_type_nan,childrens,total_salary,amount_credit,amount_annuity,region_population,...,name_yield_group_low_normal_mean,name_yield_group_middle_mean,name_yield_group_nan_mean,prev_application_number_mean_y,num_instalment_version_mean,num_instalment_number_mean,days_instalment_mean,days_entry_payment_mean,amt_instalment_mean,amt_payment_mean
0,123687442,0.0,1,0,0,1.0,157500.0,855000.0,25128.0,0.019101,...,0.000000,0.333333,0.0,49754880.5,1.0,5.75,2653.00,2659.0,6275.9250,3960.37125
1,123597908,1.0,1,0,0,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.0,49672074.0,1.0,9.00,450.00,457.0,11349.9000,11349.90000
2,123526683,0.0,1,0,0,0.0,135000.0,1006920.0,42660.0,0.026392,...,0.166667,0.166667,0.0,49376832.5,0.5,10.00,1917.25,1917.5,20169.4725,20169.47250


In [29]:
data.shape

(275234, 145)

### bki

In [30]:
bki = get_input('bki.csv')
bki.head(3)

bki.csv: 945234 rows, 17 cols


,application_number,bureau_id,credit_active,credit_currency,days_credit,credit_day_overdue,days_credit_enddate,days_enddate_fact,amt_credit_max_overdue,cnt_credit_prolong,amt_credit_sum,amt_credit_sum_debt,amt_credit_sum_limit,amt_credit_sum_overdue,credit_type,days_credit_update,amt_annuity
0,123538884,5223613,Active,currency 1,718.0,0,377.0,NaN,19386.81,0,675000.00,320265.495,0.0,0.0,Consumer credit,39.0,NaN
1,123436670,6207544,Closed,currency 1,696.0,0,511.0,511.0,0.00,0,93111.66,0.000,0.0,0.0,Consumer credit,505.0,NaN
2,123589020,6326395,Closed,currency 1,165.0,0,149.0,160.0,NaN,0,36000.00,0.000,0.0,0.0,Consumer credit,150.0,0.0


In [31]:
numerical_features = bki.select_dtypes(exclude=["object"]).columns.tolist()
categorical_features = bki.select_dtypes(include=["object"]).columns.tolist()

In [32]:
categorical_features

['credit_active', 'credit_currency', 'credit_type']

In [33]:
for feature in bki[categorical_features].columns.tolist():
    print()
    print(bki[feature].value_counts())


Closed      594315
Active      347323
Sold          3583
Bad debt        13
Name: credit_active, dtype: int64

currency 1    944492
currency 2       647
currency 3        91
currency 4         4
Name: credit_currency, dtype: int64

Consumer credit                                 688431
Credit card                                     222238
Car loan                                         15348
Mortgage                                         10161
Microloan                                         6788
Loan for business development                     1055
Another type of loan                               586
Unknown type of loan                               300
Loan for working capital replenishment             261
Cash loan (non-earmarked)                           31
Real estate loan                                    16
Loan for the purchase of equipment                  14
Loan for purchase of shares (margin lending)         4
Mobile operator loan                                

In [34]:
bki_ohe = one_hot_encoding(bki, 
                           cat_features=categorical_features, 
                           drop_original=True)

In [ ]:
data = agg_features(bki_ohe, data, by='application_number')

In [ ]:
for feature in data.columns:
    data = data.rename(columns={
        feature: get_lower(feature)
    })

In [ ]:
import re
data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [ ]:
data.head(3)

In [ ]:
data.shape

In [ ]:
data.to_pickle('data.pkl')